In [1]:
!pip install langchain langchain_community langchain-google-genai langchain_chroma langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
   

In [1]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('gemini')

In [6]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 8.1 MB/s eta 0:00:00


In [7]:
from langchain_community.document_loaders import PyPDFLoader

file_paths = [
    "/content/Financial Fraud Detection using Machine Learning Techniques.pdf",
    "/content/IJRPR16322.pdf",
    "/content/s41599-024-03606-0.pdf",
    "/content/ssrn-4895921.pdf"
]

# Combine all documents into one list
docs = []
for path in file_paths:
    loader = PyPDFLoader(path)  # Pass one file path at a time
    docs.extend(loader.load())

print(f"Loaded {len(docs)} document chunks.")
print(docs[:1])  # Preview the first chunk


Loaded 101 document chunks.
[Document(metadata={'producer': 'Prince 15.2 (www.princexml.com)', 'creator': 'Appligent pdfHarmony 2.0', 'creationdate': '2024-02-13T14:57:50-08:00', 'appligent': 'pdfHarmony 2.0 Linux Kernel 2.6 64bit Mar 13 2012 Library 9.0.1', 'author': 'Matar Al Marri and Ahmad AlAli', 'keywords': 'None provided', 'moddate': '2024-02-13T14:57:50-08:00', 'title': 'Financial Fraud Detection using Machine Learning Techniques', 'source': '/content/Financial Fraud Detection using Machine Learning Techniques.pdf', 'total_pages': 43, 'page': 0, 'page_label': '1'}, page_content='Rochester Institute of Technology Rochester Institute of Technology \nRIT Digital Institutional Repository RIT Digital Institutional Repository \nTheses \n5-2020 \nFinancial Fraud Detection using Machine Learning Techniques Financial Fraud Detection using Machine Learning Techniques \nMatar Al Marri \nmka8033@rit.edu \nAhmad AlAli \naaa4476@rit.edu \nFollow this and additional works at: https://reposito

In [8]:
print(len(docs))

101


In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

splits = text_splitter.split_documents(docs)

In [17]:
print(splits[0])

page_content='Rochester Institute of Technology Rochester Institute of Technology 
RIT Digital Institutional Repository RIT Digital Institutional Repository 
Theses 
5-2020 
Financial Fraud Detection using Machine Learning Techniques Financial Fraud Detection using Machine Learning Techniques 
Matar Al Marri 
mka8033@rit.edu 
Ahmad AlAli 
aaa4476@rit.edu 
Follow this and additional works at: https://repository.rit.edu/theses 
Recommended Citation Recommended Citation 
Al Marri, Matar and AlAli, Ahmad, "Financial Fraud Detection using Machine Learning Techniques" (2020). 
Thesis. Rochester Institute of Technology. Accessed from 
This Master's Project is brought to you for free and open access by the RIT Libraries. For more information, please 
contact repository@rit.edu.' metadata={'producer': 'Prince 15.2 (www.princexml.com)', 'creator': 'Appligent pdfHarmony 2.0', 'creationdate': '2024-02-13T14:57:50-08:00', 'appligent': 'pdfHarmony 2.0 Linux Kernel 2.6 64bit Mar 13 2012 Library 9.0.1

In [18]:
print(len(splits))

379


In [10]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [20]:
retriever = vectorstore.as_retriever(search_type= "similarity", search_kwargs={"k":6})

In [21]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature = 0.3, max_output_tokens=500)

In [23]:
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

In [24]:

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
rag_chain.invoke("How many AI courses are available?")

"This question cannot be answered from the given context. The provided text discusses financial fraud detection using machine learning, but it does not mention the number of available AI courses.  Therefore, I don't know the answer."